### Step1 Generate test cases

In [ ]:

import random
import ast

from basic_operation.dict_operation import get_rest_args
from DBOperation.dboperation_sqlite import DataBaseHandle
from combine_fragment import CodeFragmentGenerator, get_contained_cons
from cons_generator.cwvp import generate_if_cons_exist
from generate import Generate
from class_for_info.fragment_info import CodeFragmentInfo

# Load necessary tools
generate = Generate("../config.json")
fragment = CodeFragmentGenerator(1, "CodeFragment")
# Load code segment database
frag_db = DataBaseHandle(params["corpus_db"])
frag_list = frag_db.selectAll("select * from CodeFragment")
# Randomly select a code segment
frag = random.choice(frag_list)
available_variables = ast.literal_eval(frag[2])
needful_variables = ast.literal_eval(frag[3])
# Parse constraints
bool_expr_list, func_ret_list, quanternion_list, needful_args, partial_reset_stmt = \
    get_contained_cons(frag[1], needful_variables)
# print("bool_expr_list:",bool_expr_list,"func_ret_list:",func_ret_list)
if len(bool_expr_list) > 0 or len(func_ret_list) > 0:
    print("\033[94mThere exist constraints!Start to generate inputs.\033[0m")
    correct_stmt, wrong_stmt = \
        generate_if_cons_exist(bool_expr_list, func_ret_list, quanternion_list, needful_args)
    print("==Correct==\n"+correct_stmt+"\n==Wrong==\n+wrong_stmt)
else:
    correct_stmt, wrong_stmt = "//no cons\n", "//no cons\n"
# If generation failed
if correct_stmt is None:
    print("\033[91m!!!Generation failed!\033[0m")
# Generate values for other variables
needful_variables = get_rest_args(needful_variables, needful_args)
this_fragment_info = CodeFragmentInfo(frag[1], available_variables,
                                      needful_variables, frag[4], 
                                      frag[5], frag[6])
combined_fragment, combined_import = fragment.generate_a_code_frag(this_fragment_info)
if combined_fragment is None:
    print("\033[91m!!!Generation failed!\033[0m")
# Get self_defined_callables
defined_callables = ""
for item in fragment.self_defined_callables:
    if isinstance(item, str):
        if item not in defined_callables:
            defined_callables += item
    else:
        if item.content not in defined_callables:
            defined_callables += item.content
# print("===defined_callables:\n"+defined_callables+"\n")
# Process correct test case
correct_fragment = correct_stmt+combined_fragment+partial_reset_stmt
correct_import = combined_import
correct_testcase = generate.assemble_testcase(correct_fragment, correct_import, [defined_callables])
print("==correct_testcase==\n",correct_testcase)
# Process wrong test case
if wrong_stmt is None or wrong_stmt == correct_stmt:
    print("Same as below.")
else:
    wrong_fragment = wrong_stmt+combined_fragment+partial_reset_stmt
    wrong_import = combined_import
    wrong_testcase = generate.assemble_testcase(wrong_fragment, wrong_import, [defined_callables])
    print("==wrong_testcase==\n",wrong_testcase)